In [4]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:15 http://sec

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [7]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RED72VWWCOS7S|          1|            2|          8|   N|                Y|
| RZHWQ208LTEPV|          5|            0|          0|   N|                Y|
|R37LBC3XAVLYOO|          5|            4|          5|   N|                Y|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|                Y|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|                Y|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|                Y|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|                Y|
|R22C8FMBSTFRY8|          5|            2|          2|   N|                Y|
|R118NNIQ75XPGO|          3|            0|          0|   N|                Y|
|R30HYXHZQ49621|          2|            0|          0|   N|     

In [8]:
total_votes_df = vine_df.filter("total_votes >= 20")
total_votes_df.show(500)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|     

In [9]:
vote_cent_df = total_votes_df.withColumn("votes_ratio", total_votes_df["helpful_votes"] / total_votes_df["total_votes" ])
vote_cent_df.show()



+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       votes_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|0.8275862068965517|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|0.9166666666666666|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|0.9666666666666667|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|0.2857142857142857|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|0.8928571428571429|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|               1.0|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|0.8333333333333334|
|  RLNULBKRWNNR|          5|  

In [10]:
vote_cent_fifty_df = vote_cent_df.filter("votes_ratio >= 0.50")
vote_cent_fifty_df.show(50)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       votes_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|0.8275862068965517|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|0.9166666666666666|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|0.9666666666666667|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|0.8928571428571429|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|               1.0|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|0.8333333333333334|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|0.9767441860465116|
| R9QNQUL94RX1F|          3|  

In [11]:
vote_cent_fifty_df.agg({"total_votes":"sum"}).show()


+----------------+
|sum(total_votes)|
+----------------+
|         2761403|
+----------------+



In [12]:
vine_review_df = vote_cent_fifty_df.filter("vine == True") 
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       votes_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R28DXTC3JQ9IY1|          4|           24|         26|   Y|                N|0.9230769230769231|
|R3AFZKLQXATHBU|          5|           44|         49|   Y|                N|0.8979591836734694|
|R2RUUF2JPJPC0E|          4|           20|         22|   Y|                N|0.9090909090909091|
| RFZ2WUH4248AB|          2|           26|         27|   Y|                N|0.9629629629629629|
|R1Q4LVHIFOWYFR|          5|           23|         28|   Y|                N|0.8214285714285714|
| R4YEPTQED3X1Q|          5|           19|         20|   Y|                N|              0.95|
|R2Z7C8YCRSC9DP|          5|           22|         22|   Y|                N|               1.0|
|R3J8OI5CB74P5K|          1|  

In [13]:
no_vine_df = vote_cent_fifty_df.filter("vine == False") 
no_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       votes_ratio|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|0.8275862068965517|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|0.9166666666666666|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|0.9666666666666667|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|0.8928571428571429|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|               1.0|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|0.8333333333333334|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|0.9767441860465116|
| R9QNQUL94RX1F|          3|  

In [21]:
vine_total = vine_review_df.agg({"helpful_votes":"sum"}).collect()[0][0]
vine_total

23979

In [22]:
vine_five = vine_review_df.filter("star_rating == 5").agg({"helpful_votes":"sum"}).collect()[0][0]

In [24]:
vine_ratio = vine_five / vine_total
vine_ratio

0.48200508778514534

In [25]:
no_vine_total = no_vine_df.agg({"helpful_votes":"sum"}).collect()[0][0]

In [26]:
no_vine_five = no_vine_df.filter("star_rating == 5").agg({"helpful_votes":"sum"}).collect()[0][0]

In [28]:
no_vine_ratio = no_vine_five / no_vine_total
no_vine_ratio

0.5300208392243151